**Imports**

In [ ]:
import os
import psycopg2
import matplotlib.pyplot as plt

In [ ]:
mot_passe = os.environ.get('pg_psw')

**Connection + création DB**

In [ ]:
conn = psycopg2.connect(
   database="postgres", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

sql = '''CREATE database acde_manager''';

cursor.execute(sql)
print("Base de données créée avec succès !")

conn.close()

**Fonctions génériques**

In [ ]:
def ouvrir_connection(nom_bdd, utilisateur, mot_passe, host='localhost', port=5432):
    try:
        conn = psycopg2.connect(dbname=nom_bdd, user=utilisateur, password=mot_passe, host=host, port=5432)
    except psycopg2.Error as e:
        print("Erreur lors de la connection à la base de données")
        print(e)
        return None
    # On force autocommit (non applicable ds SQLite3)
    conn.set_session(autocommit=True)
    return conn


def supprimer_table(conn, sql_suppression_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_suppression_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la suppression de la table")
        print(e)
        return
    cursor.close()
    print("La table a été supprimée avec succès")

    
def creer_table(conn, sql_creation_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_creation_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("La table a été crée avec succès")

    
def inserer_donnees(conn, sql_insertion_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_insertion_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de l'insertion des données")
        print(e)
        return
    cursor.close()
    print("Les données ont été insérées avec succès")

    
def lire_donnees(conn, sql_requete):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_requete)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la lecture des données")
        print(e)
        return None
    
    print("Les données ont été lues avec succès")
    data = []
    for row in cursor:
        data.append(row)

    cursor.close()
    
    return data

def dataviz(conn, sql_dataviz):
        try:
            cursor = conn.cursor()
            cursor.execute(sql_dataviz)
            conn.commit()
        except psycopg2.Error as e:
            print("Erreur lors de la vizualisation des données")
            print(e)
            return
        
        
        ids = []
        charges = []
        for item in cursor:
            ids.append(item[0])
            charges.append(item[1])
            
        #print(charges)
        #print(ids)
        values = range(len(ids))
        print(values)
        
        # Create the plot
        plt.plot(values,charges,marker="o")
        plt.xlabel("X-Axis")
        plt.ylabel("Y-Axis")
        plt.title("Set X labels in Matplotlib Plot")
        plt.xticks(values,ids)
        plt.show()

        cursor.close()
        
        print("Les données ont été vizualisées avec succès")


**Requêtes SQL pour la table user**

In [ ]:
sql_supprimer_table_user = """
    DROP TABLE IF EXISTS users;
"""

sql_creer_table_user = """
    CREATE TABLE IF NOT EXISTS users (
    id integer primary key,
    nom text,
    prenom text,
    role text,
    equipe text,
    admin int,
    activtity text,
    email text, 
    password text,
    event_id int
    );
"""

sql_inserer_user = """
    INSERT INTO users 
    (id, nom, prenom, role, equipe, admin, activity,
    email, password, event_id)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

sql_inserer_user = """
    COPY users
    (id, nom, prenom, role, equipe, admin, activity,
    email, password, event_id)
    FROM 'C:\\Users\\Public\\Documents\\acde_users.csv'
    WITH CSV DELIMITER ','
    QUOTE '"'
    HEADER;
"""

sql_inserer_user_dict = """
    INSERT INTO users 
    (id, nom, prenom, role, equipe, admin, activity,
    email, password, event_id)
    VALUES (%(id)s, %(nom)s, %(prenom)s, %(role)s, %(equipe)s, %(admin)s, %(activity)s,
    %(email)s, %(password)s, %(event_id)s);
"""

sql_lire_user = """
    SELECT *
    FROM users;
"""

**Requêtes SQL pour la table event**

In [ ]:
sql_supprimer_table_event = """
    DROP TABLE IF EXISTS event;
"""

sql_creer_table_event = """
    CREATE TABLE IF NOT EXISTS event (
    id integer primary key,
    user_id integer,
    preparation decimal,
    duration decimal,
    charge integer,
    modalities text,
    need integer,
    participants integer,
    frequency integer,
    proprietaire text,
    date_first_event date,
    date_last_event date
    );
"""

sql_inserer_event = """
    INSERT INTO event 
    (id, user_id, preparation, duration, charge, modalities, need,
    participants, frequency, proprietaire, date_first_event, date_last_event)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

sql_inserer_event = """
    COPY event 
    (id, user_id, preparation, duration, charge, modalities, need,
    participants, frequency, proprietaire, date_first_event, date_last_event)
    FROM 'C:\\Users\\Public\\Documents\\acde_manager2.csv'
    WITH CSV DELIMITER ','
    QUOTE '"'
    HEADER;
"""

sql_inserer_event_dict = """
    INSERT INTO event 
    (id, user_id, preparation, duration, charge, modalities, need,
    participants, frequency, proprietaire, date_first_event, date_last_event)
    VALUES (%(id)s, %(user_id)s, %(preparation)s, %(duration)s, %(charge)s, %(modalities)s, %(need)s,
    %(particpants)s, %(frequency)s, %(proprietaire)s, %(date_first_event)s, %(date_last_event)s);
"""

sql_lire_event = """
    SELECT *
    FROM event
    LIMIT 10;
"""

sql_dataviz_event = """
    SELECT id, charge
    FROM event
    GROUP BY id
    LIMIT 10;
"""

sql_dataviz_event_2 = """
    SELECT id, charge
    FROM event
    GROUP BY id
    LIMIT 10 OFFSET 10;
"""

**Requêtes SQL pour la table list_of_events**

In [ ]:
sql_supprimer_table_list_of_events = """
    DROP TABLE IF EXISTS list_of_event;
"""

sql_creer_table_list_of_events = """
    CREATE TABLE IF NOT EXISTS list_of_event (
    id integer primary key,
    name_of_instance integer,
    description text,
    event_id integer
    );
"""

sql_inserer_list_of_events = """
    INSERT INTO list_of_event 
    (id, name_of_instance, description, event_id)
    VALUES (%s, %s, %s, %s);
"""

sql_inserer_list_of_events = """
    COPY list_of_event 
    id, name_of_instance, description, event_id)
    FROM 'C:\\Users\\Public\\Documents\\acde_manager.csv'
    WITH CSV DELIMITER ','
    QUOTE '"'
    HEADER;
"""

sql_inserer_list_of_events_dict = """
    INSERT INTO list_of_event 
    (id, name_of_instance, description, event_id)
    VALUES (%(id)s, %(name_of_instance)s, %(desc)s, %(event_id)s);
"""

sql_lire_list_of_events = """
    SELECT *
    FROM list_of_event;
"""

**Requêtes SQL pour la table guest**

In [ ]:
sql_supprimer_table_guest = """
    DROP TABLE IF EXISTS guest;
"""

sql_creer_table_guest = """
    CREATE TABLE IF NOT EXISTS guest (
    user_id integer,
    event_id integer
    );
"""

sql_inserer_guest = """
    INSERT INTO guest 
    (user_id, event_id)
    VALUES (%s, %s);
"""

sql_inserer_guest = """
    COPY guest 
    (user_id, event_id)
    FROM 'C:\\Users\\Public\\Documents\\acde_manager.csv'
    WITH CSV DELIMITER ','
    QUOTE '"'
    HEADER;
"""

sql_inserer_guest_dict = """
    INSERT INTO guest 
    (user_id, event_id)
    VALUES (%(user_id)s, %(event_id)s);
"""

sql_lire_guest = """
    SELECT *
    FROM guest
    LIMIT 10;
"""

**Requêtes SQL pour la table rights**

In [ ]:
sql_supprimer_table_rights = """
    DROP TABLE IF EXISTS rights;
"""

sql_creer_table_rights = """
    CREATE TABLE IF NOT EXISTS rights (
    id integer primary key,
    role integer,
    user_id integer
    );
"""

sql_inserer_rights = """
    INSERT INTO rights 
    (id, role, user_id)
    VALUES (%s, %s, %s);
"""

sql_inserer_rights = """
    COPY rights 
    (id, role, user_id)
    FROM 'C:\\Users\\Public\\Documents\\acde_manager.csv'
    WITH CSV DELIMITER ','
    QUOTE '"'
    HEADER;
"""

sql_inserer_rights_dict = """
    INSERT INTO rights 
    (id, role, user_id)
    VALUES (%(id)s, %(role)s, %(user_id)s);
"""

sql_lire_rights = """
    SELECT *
    FROM rights;
"""

**Parametres bdd**

In [ ]:
ma_base_donnees = "acde_manager"
utilisateur = "postgres"
mot_passe = os.environ.get('pg_psw')

**Connect BDD**

In [ ]:
conn = ouvrir_connection(ma_base_donnees, utilisateur, mot_passe)

type(conn)

**Create tables**

In [ ]:
creer_table(conn, sql_creer_table_user)

In [ ]:
creer_table(conn, sql_creer_table_event)

In [ ]:
creer_table(conn, sql_creer_table_list_of_events)

In [ ]:
creer_table(conn, sql_creer_table_guest)

In [ ]:
creer_table(conn, sql_creer_table_rights)

**Import CSV**

In [ ]:
inserer_donnees(conn, sql_inserer_user)

In [ ]:
inserer_donnees(conn, sql_inserer_event)

In [ ]:
inserer_donnees(conn, sql_inserer_list_of_events)

In [ ]:
inserer_donnees(conn, sql_inserer_guest)

In [ ]:
inserer_donnees(conn, sql_inserer_rights)

**Lecture data**

In [ ]:
lire_donnees(conn, sql_lire_user)

In [ ]:
lire_donnees(conn, sql_lire_event)

In [ ]:
lire_donnees(conn, sql_lire_list_of_events)

In [ ]:
lire_donnees(conn, sql_lire_guest)

In [ ]:
lire_donnees(conn, sql_lire_rights)

**Delete tables**

In [ ]:
supprimer_table(conn, sql_supprimer_table_user)

In [ ]:
supprimer_table(conn, sql_supprimer_table_event)

In [ ]:
supprimer_table(conn, sql_supprimer_table_list_of_events)

In [ ]:
supprimer_table(conn, sql_supprimer_table_guest)

In [ ]:
supprimer_table(conn, sql_supprimer_table_rights)

**matplotlib dataviz event**

In [ ]:
lire_donnees(conn, sql_dataviz_event)

In [ ]:
dataviz(conn, sql_dataviz_event)

In [ ]:
dataviz(conn, sql_dataviz_event_2)